#### Calculating Dice Coefficient

In [1]:
import numpy as np
import glob

from utils import load_images_array
from dice_ev import dice_coef

ModuleNotFoundError: No module named 'image'

In [ ]:
TEMPO = []
ORIGINAL_SIZE = 850
NEW_SIZE = 256

# Choose train folder TM40 ou TM46
_folder = './TM40_Original'

GT_imgs = sorted(glob.glob(_folder + '/GT_images/*'))
pred_imgs = sorted(glob.glob('predict_path/*'))

gt = load_images_array(GT_imgs, new_size = NEW_SIZE)
pred = load_images_array(pred_imgs, new_size = NEW_SIZE)

In [ ]:
print(type(pred)) 

In [ ]:
gt = np.stack(gt)
pred = np.stack(pred)

In [ ]:
print(type(pred)) # Verificar se há mudança de tipo, caso não, o stack pode ser retirado

In [ ]:
dice = dice_coef(gt, pred)
print(dice)
#dc1 = dice_coef(y_new.astype('float32'), y_pred1.astype('float32'))